In [1]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import os
from os import listdir
import pickle as pk
import json
import numpy as np

In [2]:
def load_from_json(file:str):
    return json.loads(open(f'embeddings/{file}').read())

In [3]:
from sklearn.neighbors import KNeighborsClassifier
def train_knn_and_test(features, labels, sample_for_score=500):
    start = time.time()
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(features,labels)
    return knn,knn.score(features[:sample_for_score], labels[:sample_for_score]), time.time()-start

In [4]:
from sklearn.decomposition import PCA
def train_pca(train_set, percentage=90):
    for i in range(50,min(sample_size,4096),50):
        pca = PCA(n_components=i)
        pca.fit(train_set)
        if np.sum(100*pca.explained_variance_ratio_) > percentage:
            return i,pca

## Training with and without pca

0.718 accuracy sur 1000 examples 10nn 20k

In [5]:
import time

In [8]:
listdir('./embeddings')

['embeddingsresnet50_20000.json',
 'embeddingsresnet50_30000.json',
 'embeddingsresnet50_50000.json',
 '.ipynb_checkpoints',
 'embeddingsvgg16_3000.json',
 'embeddingsresnet50_10000.json']

In [11]:
j = 0
df = pd.DataFrame(columns=['PCA','Train accuracy','Sample size', 'Execution time'])
for file in listdir('./embeddings'):
    if '.json' in file and 'resnet' in file:
        sample_size = int(file[file.index('_')+1:file.index('.')])
        print(sample_size)
        [embeddings, labels] = load_from_json(file)

        # Train knn
        knn, knn_score,tme = train_knn_and_test(embeddings, labels)
        
        df.loc[j] = [0,knn_score, sample_size, tme]
        j += 1
        # Fit pca
        i,pca = train_pca(embeddings)
        print(f'Trained pca with {i} components')
        
        # Train knn
        knn, knn_score, tme = train_knn_and_test(pca.transform(embeddings),labels)
        df.loc[j] = [1,knn_score,sample_size, tme]
        j+=1
        print(df)
df
        

20000
Trained pca with 250 components
   PCA  Train accuracy  Sample size  Execution time
0  0.0           0.718      20000.0       41.338886
1  1.0           0.708      20000.0        5.888561
30000
Trained pca with 250 components
   PCA  Train accuracy  Sample size  Execution time
0  0.0           0.718      20000.0       41.338886
1  1.0           0.708      20000.0        5.888561
2  0.0           0.740      30000.0       58.010997
3  1.0           0.744      30000.0        7.683334
50000
Trained pca with 250 components
   PCA  Train accuracy  Sample size  Execution time
0  0.0           0.718      20000.0       41.338886
1  1.0           0.708      20000.0        5.888561
2  0.0           0.740      30000.0       58.010997
3  1.0           0.744      30000.0        7.683334
4  0.0           0.766      50000.0      101.989553
5  1.0           0.766      50000.0       13.048629
10000
Trained pca with 250 components
   PCA  Train accuracy  Sample size  Execution time
0  0.0          

,PCA,Train accuracy,Sample size,Execution time
0,0.0,0.718,20000.0,41.338886
1,1.0,0.708,20000.0,5.888561
2,0.0,0.740,30000.0,58.010997
3,1.0,0.744,30000.0,7.683334
4,0.0,0.766,50000.0,101.989553
5,1.0,0.766,50000.0,13.048629
6,0.0,0.708,10000.0,20.843700
7,1.0,0.716,10000.0,2.983448


In [16]:
df.sort_values(by="Train accuracy", ascending=False)

,PCA,Train accuracy,Sample size,Execution time
4,0.0,0.766,50000.0,101.989553
5,1.0,0.766,50000.0,13.048629
3,1.0,0.744,30000.0,7.683334
2,0.0,0.740,30000.0,58.010997
0,0.0,0.718,20000.0,41.338886
7,1.0,0.716,10000.0,2.983448
1,1.0,0.708,20000.0,5.888561
6,0.0,0.708,10000.0,20.843700


In [17]:
df.sort_values(by="Execution time", ascending=False)

,PCA,Train accuracy,Sample size,Execution time
4,0.0,0.766,50000.0,101.989553
2,0.0,0.740,30000.0,58.010997
0,0.0,0.718,20000.0,41.338886
6,0.0,0.708,10000.0,20.843700
5,1.0,0.766,50000.0,13.048629
3,1.0,0.744,30000.0,7.683334
1,1.0,0.708,20000.0,5.888561
7,1.0,0.716,10000.0,2.983448


In [207]:
confusion_matrix(knn.predict(pca.transform(feats_dev)), labels_dev)
#['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

array([[23,  6,  4,  3,  2,  2,  3,  2,  6,  4],
       [ 4, 23,  6,  2,  2,  3,  3,  4,  4,  8],
       [ 3,  2, 13,  3,  4,  2,  2,  2,  0,  0],
       [ 2,  4,  7, 15,  3, 12,  3,  2,  4,  2],
       [ 1,  0, 12, 11, 24,  9, 17, 10,  4,  0],
       [ 5,  5,  3, 10,  4, 15,  6,  9,  1,  2],
       [ 1,  5,  9,  8,  5,  4, 28,  0,  1,  2],
       [ 2,  5,  3,  0,  8,  3,  2, 13,  2,  7],
       [17,  1,  0,  0,  1,  1,  1,  1, 40,  1],
       [ 6, 12,  1,  3,  3,  2,  4,  3,  5, 43]])

In [68]:
from sklearn.manifold import TSNE
embeddings = np.array(load_from_json('embeddings_3000.json'))


## Training with and without t-SNE